<a href="https://colab.research.google.com/github/algo21-221040114/PytorchTest/blob/main/PytorchLightning_complex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
pip install pytorch_lightning

In [ ]:
pip install torchmetrics

In [4]:
from torchmetrics.functional import accuracy

import pytorch_lightning
from pytorch_lightning import LightningModule, Trainer

In [15]:
# Pytorch Lightning
# reference: https://zhuanlan.zhihu.com/p/410394574
class PLModel(LightningModule):
  
  def __init__(self, full_data, test_data, channels, height, width, hidden_size, batch_size):
    super().__init__()

    self.full_data = full_data
    self.test_data = test_data
    self.channels = channels
    self.height = height
    self.width = width
    self.hidden_size = hidden_size
    self.batch_size = batch_size

    self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(self.channels * self.height * self.width, self.hidden_size*2),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(self.hidden_size*2, self.hidden_size),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(self.hidden_size, 1)
    )

  def forward(self, x):
    x = self.model(x)
    return x

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
    return optimizer

  def training_step(self, batch, batch_idx):
    x, y = batch
    pred_y = self.model(x)
    loss_fn = nn.MSELoss(reduce=True, size_average=True)
    loss = loss_fn(pred_y.float(), y.float())
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    pred_y = self.model(x)
    loss_fn = nn.MSELoss(reduce=True, size_average=True)
    loss = loss_fn(pred_y.float(), y.float())

    self.log('val_loss', loss, prog_bar=True)
    return loss
    
  def test_step(self, batch, batch_idx):
      return self.validation_step(batch, batch_idx)

  def setup(self, stage=None):

      # Assign train/val datasets for use in dataloaders
      if stage == 'fit' or stage is None:
          self.train1, self.val1 = random_split(self.full_data, [90, 10])

      # Assign test dataset for use in dataloader(s)
      if stage == 'test' or stage is None:
          self.test1 = self.test_data


  def train_dataloader(self):
      return DataLoader(self.train1, batch_size=self.batch_size)

  def val_dataloader(self):
      return DataLoader(self.val1, batch_size=self.batch_size)

  def test_dataloader(self):
      return DataLoader(self.test1, batch_size=self.batch_size)

In [8]:
# Process data
train_feature = torch.rand((100, 3, 16, 16))
train_label = torch.rand((100, 1))
torch_dataset1 = TensorDataset(train_feature, train_label)
test_feature = torch.rand((10, 3, 16, 16))
test_label = torch.rand((10, 1))
torch_dataset2 = TensorDataset(test_feature, test_label)

In [16]:
model = PLModel(full_data = torch_dataset1,
                test_data = torch_dataset2,
                channels=3, 
                height=16, 
                width=16, 
                hidden_size=5, 
                batch_size=8)

In [17]:
trainer = Trainer(
    accelerator="auto", # 使用accelerator来选择将要训练的硬件，可设置cpu、gpu、tpu以及自动选择
    max_epochs=50)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(model)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 7.8 K 
-------------------------------------
7.8 K     Trainable params
0         Non-trainable params
7.8 K     Total params
0.031     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [ ]:
# 全局变量
AVAIL_GPUS = min(1, torch.cuda.device_count())